In [55]:
import sklearn as sk
import scipy as sp
import mne
import numpy as np

In [56]:
#Load a mat file and corresponding metadata
example_mat = sp.io.loadmat('data/20110706/LFP_ch1.mat')
example_time_info = sp.io.loadmat('data/20110706/Movie_start_time.mat')
example_task_info = sp.io.loadmat('data/20110706/Task_info.mat')

example_mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI, Created on: Thu Jul  7 19:42:14 2011',
 '__version__': '1.0',
 '__globals__': [],
 'LFP': array([[-136, -123, -122, ...,  -38,  -20,   -5]], dtype=int16)}

In [57]:
#load a mat file into mne
'''
we should be able to directly load a mat file with something like this, but I couldn't get it to work
    example_eeg = mne.io.read_raw_fieldtrip('data/20110706/LFP_ch1.mat', info=None, data_name='LFP')
instead I used this tutorial
https://mne.tools/stable/auto_tutorials/simulation/10_array_objs.html
'''

#create info
n_channels = 128
sampling_freq = 1000
info = mne.create_info(n_channels, sfreq=sampling_freq)

#load the data from the scipy loaded mat:
#data would be a 2d np array where each row is loaded from one of the mat files as above
data = DUMMY
raw = mne.io.RawArray(data, info)
raw.plot()


NameError: name 'DUMMY' is not defined